# Separating categories into groups with less than n records

This notebook uses python code to split categories from Wos/Scopus databases into groups with less than n records, in order to facilitate retrieval of bibliographic information.

The base files with categories/record count information where obtained using the 'analyze' option of each database.

## Defining main functions

In [1]:
import pandas as pd

In [2]:
def get_sorted_df(df, col_index, ascend=False):
    sorted_indexes = df.iloc[:,col_index].sort_values(ascending=ascend).index #Sorting by column at 'col_index' position and obtaining sorted indexes list
    sorted_df = df.iloc[sorted_indexes,:].dropna() #Obtaining sorted dataframe (without na values)
    return sorted_df

def get_groups_under_n_records(df, max_records=1000):
    '''Receives a df with two fields: The first is Category and the second is Record Count (2nd column will be sorted by decreasing values)
    Iterates over each row, while keeping a counter variable.
    If record count > max_record, it adds the category as a dictionary entry.
    Else, record count from multiple categories are added to counter, until it goes over the max_record limit.
    The categories that 'fit' within the max_record limit will be added as a dictionary entry.
    Meanwhile, the category that 'went over' said limit will reset the counter variable and start a new group.
    In the end, we'll have a nested dictionary: {group_number : {categories: [list_of_cats], records: n_records_in_group}}
    e.g.
    { 18: {'cat': ['Multidisciplinary', 'Nursing'],  'recs': 1745} }
    '''
    n_group = 1 #Start first group
    counter = 0 #Start counter variable
    groups = {} #Start groups dictionary
    df = get_sorted_df(df, 1, ascend=False) #Sorting by 2nd column (descending)
    for index, row in df.iterrows():
        (category, records) = row
        if records > max_records: #Deals exclusively with categories exceeding the max_records limit, does not update counter var
            groups[n_group] = {} 
            groups[n_group]["cat"] = []
            groups[n_group]["cat"].append(category)
            groups[n_group]["recs"] = records
            #groups[n_group]["split"] = True #Adds a 'split' key to categories that exceed the max_record
            n_group += 1
            continue #Goes immediately to next row
        counter += records
        if counter > max_records: #If a given category makes the counter var "go over" max_limit, resets counter variable and starts a new group with said category
            n_group += 1 #starts a new group
            groups[n_group] = {}
            groups[n_group]["cat"] = [] #Adds category to new group
            groups[n_group]["cat"].append(category)
            groups[n_group]["recs"] = records 
            counter = records #New counter value is the number of records from the category
        else: #If counter variable did not surpass max_value, will update cat and recs
            if not groups.get(n_group): #Just in case the group has not been created yet (e.g. First column has less records than max_record limit)
                groups[n_group] = {}
                groups[n_group]["cat"] = []
            groups[n_group]["cat"].append(category) #Updates categories list
            groups[n_group]["recs"] = counter #Updates counter variable
    return groups

In [3]:
def get_cat_group_df(groups_dict):
    '''Obtaining a list of tuples:
    [(category_1, download_group_1), ... (category_n, download_group_n)]
    This list will be used to generate a new dataframe using pd.DataFrame.from_records(),
    which will be returned from the function'''
    cat_group_list = []
    for key in groups_dict.keys():
        for value in groups_dict.get(key).get('cat'):
            cat_group_list.append((value, key, groups_dict.get(key).get('recs'))) 
    groups_df = pd.DataFrame.from_records(cat_group_list, columns=['Category', 'Download Group', 'Group Records'])
    return groups_df

## Getting groups csv file for Scopus

In [8]:
scopus = pd.read_csv('scopus_categories_ufrj_2017-2021.csv', skiprows=8, names=['Category','Record Count'])
groups_dict = get_groups_under_n_records(scopus, max_records=2000)
groups_df = get_cat_group_df(groups_dict)
display(groups_df)
groups_df.iloc[:,0:2].to_csv("scopus_download_groups.csv", index=False) #We want only the first two columns

,Category,Download Group,Group Records
0,Medicine,1,6294
1,Agricultural and Biological Sciences,2,3574
2,Engineering,3,3464
3,Chemistry,4,3151
4,"Biochemistry, Genetics and Molecular Biology",5,3125
5,Physics and Astronomy,6,3040
6,Social Sciences,7,2518
7,Environmental Science,8,2382
8,Materials Science,9,1886
9,Computer Science,10,1879


## Getting groups csv file for WoS

In [15]:
wos = pd.read_csv('wos_categories_ufrj_2017-2021.tsv', delimiter='\t', usecols=[0,1]) #Only first 2 columns ("Category" and "Records Count") are needed
groups_dict = get_groups_under_n_records(wos, max_records=500)
groups_df = get_cat_group_df(groups_dict)
#groups_df.iloc[:,0:2].to_csv("wos_download_groups.csv", index=False) #We want only the first two columns
groups_df.to_csv("wos_download_groups_no_nan.csv", index=False)

In [16]:
groups_df['Split by Year'] = groups_df['Group Records'] > 500
groups_df

,Category,Download Group,Group Records,Split by Year
0,Environmental Sciences,1,892.0,True
1,Biochemistry Molecular Biology,2,826.0,True
2,Physics Particles Fields,3,748.0,True
3,Multidisciplinary Sciences,4,742.0,True
4,Zoology,5,707.0,True
...,...,...,...,...
241,Psychology Educational,82,96.0,False
242,Psychology Mathematical,82,96.0,False
243,Andrology,82,96.0,False
244,Dance,82,96.0,False


## Problem with group 81 from Wos

The number of WoS files (468) does not match the counter of the function I wrote (493) in category group 81.

I made a [query](https://www.webofscience.com/wos/woscc/summary/479e0f48-3d7b-4991-8963-830b32763a42-423cb830/relevance/1) with all categories from group 81 and downloaded the 'analyze.txt' file.




In [45]:
#Getting all "group 81" categories (selected them by visual inspection of table)
group81 = wos.iloc[191:223,:]
display(group81)
display(len(group81))
group81['Record Count'].sum()
#wos.tail(60)

,Web of Science Categories,Record Count
191,Robotics,25.0
192,International Relations,24.0
193,Agriculture Dairy Animal Science,22.0
194,Medicine Legal,22.0
195,Psychology Experimental,22.0
196,Soil Science,21.0
197,Urology Nephrology,20.0
198,Otorhinolaryngology,19.0
199,Ethics,18.0
200,Ergonomics,17.0


32

493.0

In [62]:
#Getting categories from "group 81" as list
cats81 = group81.iloc[:,0].to_list()
print(cats81, len(cats81))

['Robotics', 'International Relations', 'Agriculture Dairy Animal Science', 'Medicine Legal', 'Psychology Experimental', 'Soil Science', 'Urology Nephrology', 'Otorhinolaryngology', 'Ethics', 'Ergonomics', 'Literary Theory Criticism', 'Materials Science Ceramics', 'Microscopy', 'Quantum Science Technology', 'Regional Urban Planning', 'Psychology Biological', 'Social Sciences Biomedical', 'Audiology Speech Language Pathology', 'Ornithology', 'Social Sciences Mathematical Methods', 'Cultural Studies', 'Materials Science Textiles', 'Ethnic Studies', 'Logic', 'Psychology Developmental', 'Psychology Social', 'Social Work', 'Engineering Aerospace', 'Primary Health Care', 'Family Studies', 'Psychology Applied', 'Social Issues'] 32


In [47]:
#Reading the group81.tsv (obtained from query with only group 81 categories)
test = pd.read_csv('group81.tsv', delimiter='\t')
test.head()

,Web of Science Categories,Record Count,% of 468
0,Robotics,25,5.342
1,Computer Science Artificial Intelligence,24,5.128
2,International Relations,24,5.128
3,Agriculture Dairy Animal Science,22,4.701
4,Medicine Legal,22,4.701


In [63]:
#Got only rows where category is in the cats81 list, then summed its Records and %
test[test.iloc[:,0].isin(cats81)].iloc[:,1:].sum(axis=0)

Record Count    493.000
% of 468        105.338
dtype: float64

In [64]:
# Calculating number of actual elements based on record count and %
493 * (100 / 105.338) 

468.01723974254304

In [ ]:
# Thus, the REAL number of unique elements is, in fact, 468.
# The additional 25 publications probably had multiple categories, some of them in common

## Is there a way to minimize the number of groups?

- Yes, it's an common optimization problem called *[bin packing problem](https://en.wikipedia.org/wiki/Bin_packing_problem)*

In [112]:
import binpacking
import pandas as pd

In [233]:
def create_dict_from_series(discipline,n_records):
    '''
    Takes two series (discipline name and number of records, respectively)
    and returns a dictionary with those two values).
    Data needs to be in a dictionary for the binpacking module.
    '''
    recs_dict = dict(zip(discipline, n_records))
    return recs_dict

def get_bins(binned):
    bins = {count: entry for count, entry in enumerate(binned, start=1)} #Get categories in each bin
    melted = []
    for i in bins:
        for key, value in bins.get(i).items():
            melted.append((i, key, value))
    return melted
            
def get_df(bins):
    df = pd.DataFrame(bins, columns=['group','discipline', 'n_recs']) #Name of columns can be changed here
    df['group_total'] = df.groupby('group')['n_recs'].transform(sum) #Calculating number of records for group
    return df
    
def binpacking_records(recs_dict, max_records):
    binned = binpacking.to_constant_volume(recs_dict,max_records) #Use binpacking algorithm
    bins = get_bins(binned) # Melts the data (when converted to df, will yield one collumn per discipline)
    df = get_df(bins) #Converts to dataframe 
    return df

## Getting download groups for WoS

In [237]:
wos = pd.read_csv('wos_categories_ufrj_2010-2021_31-10-2022.csv',
                  sep='\t',
                  usecols=[0,1]).dropna() #dropna removes unclassified documents (normally a insignificant ammount of records)

In [238]:
wos_dict = create_dict_from_series(wos.iloc[:,0], wos.iloc[:,1]) #Getting a dictionary (input for the 'binpacking' package) based on 'analyse' results

In [239]:
wos_df = binpacking_records(wos_dict, 500)
wos_df

,group,discipline,n_recs,group_total
0,1,Biochemistry Molecular Biology,1972.0,1972.0
1,2,Physics Particles Fields,1753.0,1753.0
2,3,Zoology,1607.0,1607.0
3,4,Multidisciplinary Sciences,1573.0,1573.0
4,5,Neurosciences,1572.0,1572.0
...,...,...,...,...
242,106,Psychology Applied,11.0,499.0
243,106,Agricultural Economics Policy,8.0,499.0
244,106,Emergency Medicine,8.0,499.0
245,107,Film Radio Television,8.0,14.0


In [240]:
wos_df.to_csv('wos_binpacking.csv', index=False)

## Getting download groups for Scopus

In [241]:
#First lines (query metadata) were manually removed
scopus = pd.read_csv('Scopus-56919-Analyze-Subject.csv',
                  sep=',',
                  usecols=[0,1]).dropna() #dropna removes unclassified documents (normally a insignificant ammount of records)

In [242]:
scopus_dict = create_dict_from_series(scopus.iloc[:,0], scopus.iloc[:,1]) #Getting a dictionary (input for the 'binpacking' package) based on 'analyse' results

In [246]:
scopus_df = binpacking_records(scopus_dict, 2000)
scopus_df

,group,discipline,n_recs,group_total
0,1,Medicine,13344,13344
1,2,Agricultural and Biological Sciences,7283,7283
2,3,Engineering,7002,7002
3,4,Physics and Astronomy,6947,6947
4,5,"Biochemistry, Genetics and Molecular Biology",6751,6751
5,6,Chemistry,6542,6542
6,7,Social Sciences,4560,4560
7,8,Environmental Science,4253,4253
8,9,Computer Science,4229,4229
9,10,Materials Science,3685,3685


In [247]:
scopus_df.to_csv('scopus_binpacking.csv', index=False)